In [1]:
# Run the enhanced water risk analysis
from water_risk_scorer import run_risk_analysis, test_schnitzer_well

# First, run the unit test to verify the enhanced methodology
print("=== RUNNING UNIT TEST ===")
test_result = test_schnitzer_well()

if test_result:
    print("\n=== RUNNING FULL ENHANCED ANALYSIS ===")
    final_results_df, gis_data = run_risk_analysis('wells_input.csv')
    
    if final_results_df is not None:
        print(f"\n✅ Analysis completed successfully for {len(final_results_df)} wells")
        
        # Display enhanced results
        display_cols = [
            'WELL_NAME', 'COUNTY', 'final_score', 'risk_tier', 
            'P_Leak', 'Domestic_Demand_Wtd_m3_yr', 
            'Water_Safeguarded_m3_yr', 'Water_Safeguarded_acft_yr'
        ]
        
        print("\n📊 Enhanced Water Safeguarded Results:")
        print(final_results_df[display_cols].to_string(index=False))
        
        total_safeguarded = final_results_df['Water_Safeguarded_acft_yr'].sum()
        print(f"\n🌊 Total Enhanced Water Safeguarded: {total_safeguarded:.2f} acre-feet/year")
        
        # Generate outputs
        from run_analysis import save_outputs, generate_maps
        save_outputs(final_results_df)
        generate_maps(final_results_df, gis_data)
        print("\n💾 All outputs saved to 'output' directory")
    else:
        print("❌ Analysis failed")
else:
    print("❌ Unit test failed - cannot proceed")


=== RUNNING UNIT TEST ===

=== UNIT TEST: SCHNITZER #2 ===
Score: 67
Domestic Demand: 600 m³/yr
DRASTIC Factor: 0.8 (High)
Calculated P_Leak: 0.725 (Expected: ~0.73)
Calculated Water Safe: 435 m³/yr (Expected: ~439 m³)
✅ Unit test PASSED!

=== RUNNING FULL ENHANCED ANALYSIS ===
Loaded and processed 5 wells.

Loading enhanced data sources...

'gw_owrb_aquifers.shp' already exists. Skipping download.
'Shape/NHDFlowline_0.shp' already exists. Skipping download.

Loading GIS layers into memory...
GIS data loaded successfully.

--- Processing well 3508320686 (BLAKE) ---
  Enhanced Water Safeguarded: 347.0 m³/yr (0.281 ac-ft/yr)
  DRASTIC Factor: 0.8 (High)
  Leak Probability: 0.226
  Distance-Weighted Demand: 1536.8 m³/yr

--- Processing well 3503921266 (JUDITH) ---
  Enhanced Water Safeguarded: 0.0 m³/yr (0.0 ac-ft/yr)
  DRASTIC Factor: 0.8 (High)
  Leak Probability: 0.031
  Distance-Weighted Demand: 0.0 m³/yr

--- Processing well 3506121229 (KING-OH) ---
  Enhanced Water Safeguarded: 4770

In [ ]:
# Import required libraries
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import warnings
from scipy.stats import truncnorm
warnings.filterwarnings('ignore')

# Configuration parameters
TARGET_CRS = 'EPSG:3857'
GEOGRAPHIC_CRS = 'EPSG:4326'
DATA_DIR = 'data'
OUTPUT_DIR = 'output'

# Enhanced Water Safeguarded Parameters
LEAK_RATE_RANGE_M3_DAY = (0.5, 5.9)  # Range of potential leak rates
RUN_MONTE_CARLO = False               # Toggle for Monte Carlo simulation
MONTE_CARLO_ITERATIONS = 10000        # Number of Monte Carlo iterations

# DRASTIC Vulnerability Mappings
DRASTIC_MAPPING = {
    'Very High': 1.0,
    'High': 0.8,
    'Moderate': 0.6,
    'Low': 0.4,
    'Very Low': 0.2
}

print("✅ Configuration loaded successfully")


In [ ]:
def get_drastic_factor(well_pt, drastic_ds=None):
    """
    Returns DRASTIC vulnerability multiplier (1.0 for Very High ... 0.2 for Very Low).
    In full implementation, would integrate actual DRASTIC raster data.
    """
    # Placeholder implementation - would integrate real DRASTIC data
    # For demonstration, assigning based on well characteristics
    return 0.8  # Default to 'High' vulnerability

def get_county_water_use_data():
    """
    Mock function to get county-level water use data.
    In full implementation, would download USGS County-Level Water Use CSV.
    """
    mock_data = {
        'ALFALFA': {'self_supplied_domestic_m3': 2500000, 'domestic_wells': 850},
        'HASKELL': {'self_supplied_domestic_m3': 1800000, 'domestic_wells': 600},
        'LOGAN': {'self_supplied_domestic_m3': 3200000, 'domestic_wells': 1100},
        'GARFIELD': {'self_supplied_domestic_m3': 4500000, 'domestic_wells': 1500},
        'CUSTER': {'self_supplied_domestic_m3': 2800000, 'domestic_wells': 950}
    }
    return pd.DataFrame.from_dict(mock_data, orient='index').reset_index()

def distance_weighted_demand(well_pt, domestic_wells_gdf, county_use_df, county_name):
    """
    Calculate distance-weighted domestic water demand.
    Σ (annual_use_per_well * weight) where weight = max(0, 1 - d/1000)
    """
    if domestic_wells_gdf is None or len(domestic_wells_gdf) == 0:
        return 0.0
    
    # Get county-specific annual use per well
    county_data = county_use_df[county_use_df['index'] == county_name]
    if len(county_data) == 0:
        annual_use_per_well = 300  # fallback to default
    else:
        county_row = county_data.iloc[0]
        annual_use_per_well = county_row['self_supplied_domestic_m3'] / county_row['domestic_wells']
    
    # Calculate distance weights and sum
    distances = domestic_wells_gdf.distance(well_pt)
    weights = np.maximum(0, 1 - distances/1000)  # 1km cutoff
    weighted_demand = np.sum(annual_use_per_well * weights)
    
    return weighted_demand

print("✅ Feature engineering functions defined")


In [ ]:
def sigmoid_prob(score):
    """
    Probability model: sigmoid function to convert risk score to leak probability.
    """
    return 1 / (1 + np.exp(-(score - 50)/7.5))

def monte_carlo_water_safeguarded(dom_demand, p_leak, iterations=MONTE_CARLO_ITERATIONS):
    """
    Monte Carlo simulation for uncertainty quantification.
    Returns percentiles (p5, p50, p95) of water safeguarded estimates.
    """
    # Simulate uncertainty in demand (±20%)
    demand_std = dom_demand * 0.2
    demand_samples = np.random.normal(dom_demand, demand_std, iterations)
    demand_samples = np.maximum(demand_samples, 0)  # ensure positive
    
    # Simulate uncertainty in probability (Beta distribution)
    alpha = p_leak * 10 + 1
    beta = (1 - p_leak) * 10 + 1
    prob_samples = np.random.beta(alpha, beta, iterations)
    
    # Calculate water safeguarded for each iteration
    water_safe_samples = demand_samples * prob_samples
    
    return {
        'p5': np.percentile(water_safe_samples, 5),
        'p50': np.percentile(water_safe_samples, 50),
        'p95': np.percentile(water_safe_samples, 95)
    }

# Demonstrate probability model
scores = np.arange(0, 101, 10)
probs = [sigmoid_prob(s) for s in scores]

plt.figure(figsize=(10, 6))
plt.plot(scores, probs, 'b-', linewidth=2, label='Leak Probability')
plt.xlabel('Risk Score')
plt.ylabel('Leak Probability')
plt.title('Sigmoid Probability Model: Risk Score → Leak Probability')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

print("✅ Probability model and Monte Carlo functions defined")


In [ ]:
def test_schnitzer_well():
    """
    Unit test for SCHNITZER #2 well as specified in requirements.
    """
    print("=== UNIT TEST: SCHNITZER #2 ===")
    
    # Mock well data
    score = 67
    dom_demand_mock = 600  # m³/yr
    drastic_factor = 0.8   # High vulnerability
    
    # Test sigmoid probability
    p_leak = sigmoid_prob(score) * drastic_factor
    expected_p_leak = 0.73  # ±0.01
    
    # Test water safeguarded
    water_safe = dom_demand_mock * p_leak
    expected_water_safe = 439  # ±5 m³
    
    print(f"Score: {score}")
    print(f"Domestic Demand: {dom_demand_mock} m³/yr")
    print(f"DRASTIC Factor: {drastic_factor} (High)")
    print(f"Calculated P_Leak: {p_leak:.3f} (Expected: ~0.73)")
    print(f"Calculated Water Safe: {water_safe:.0f} m³/yr (Expected: ~439 m³)")
    
    # Assertions (allow small tolerance for floating point precision)
    try:
        assert abs(p_leak - expected_p_leak) <= 0.01, f"P_Leak test failed: {p_leak} vs {expected_p_leak}"
        assert abs(water_safe - expected_water_safe) <= 5, f"Water Safe test failed: {water_safe} vs {expected_water_safe}"
        print("✅ Unit test PASSED!")
        return True
    except AssertionError as e:
        print(f"❌ Unit test FAILED: {e}")
        return False

# Run the unit test
test_result = test_schnitzer_well()


In [ ]:
# Import the enhanced analysis from our module
from water_risk_scorer import run_risk_analysis

# Run the enhanced water risk analysis
print("Starting Enhanced Water Risk Analysis...")
final_results_df, gis_data = run_risk_analysis('wells_input.csv')

if final_results_df is not None:
    print(f"\n✅ Analysis completed successfully for {len(final_results_df)} wells")
    
    # Display key results
    display_cols = [
        'WELL_NAME', 'COUNTY', 'final_score', 'risk_tier', 
        'Drastic_Class', 'P_Leak', 'Domestic_Demand_Wtd_m3_yr', 
        'Water_Safeguarded_m3_yr', 'Water_Safeguarded_acft_yr'
    ]
    
    print("\n📊 Enhanced Water Safeguarded Results:")
    print(final_results_df[display_cols].to_string(index=False))
else:
    print("❌ Analysis failed")


In [ ]:
# Enable Monte Carlo simulation for uncertainty quantification
RUN_MONTE_CARLO = True

if RUN_MONTE_CARLO and final_results_df is not None:
    print("🎲 Running Monte Carlo simulation for uncertainty quantification...")
    
    # Demonstrate Monte Carlo for highest-risk well (KING-OH)
    king_oh = final_results_df[final_results_df['WELL_NAME'] == 'KING-OH'].iloc[0]
    
    dom_demand = king_oh['Domestic_Demand_Wtd_m3_yr']
    p_leak = king_oh['P_Leak']
    
    if dom_demand > 0:
        mc_results = monte_carlo_water_safeguarded(dom_demand, p_leak)
        
        print(f"\n📊 Monte Carlo Results for {king_oh['WELL_NAME']}:")
        print(f"  Deterministic: {king_oh['Water_Safeguarded_m3_yr']:.0f} m³/yr")
        print(f"  5th percentile: {mc_results['p5']:.0f} m³/yr")
        print(f"  50th percentile (median): {mc_results['p50']:.0f} m³/yr")
        print(f"  95th percentile: {mc_results['p95']:.0f} m³/yr")
        
        # Visualize uncertainty
        plt.figure(figsize=(10, 6))
        
        # Generate sample for histogram
        iterations = 1000
        demand_std = dom_demand * 0.2
        demand_samples = np.random.normal(dom_demand, demand_std, iterations)
        demand_samples = np.maximum(demand_samples, 0)
        
        alpha = p_leak * 10 + 1
        beta = (1 - p_leak) * 10 + 1
        prob_samples = np.random.beta(alpha, beta, iterations)
        
        water_safe_samples = demand_samples * prob_samples
        
        plt.hist(water_safe_samples, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
        plt.axvline(mc_results['p5'], color='red', linestyle='--', label=f'5th percentile: {mc_results["p5"]:.0f}')
        plt.axvline(mc_results['p50'], color='green', linestyle='-', label=f'Median: {mc_results["p50"]:.0f}')
        plt.axvline(mc_results['p95'], color='red', linestyle='--', label=f'95th percentile: {mc_results["p95"]:.0f}')
        
        plt.xlabel('Water Safeguarded (m³/yr)')
        plt.ylabel('Frequency')
        plt.title(f'Monte Carlo Uncertainty Analysis: {king_oh["WELL_NAME"]} Water Safeguarded')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()
    else:
        print("No domestic demand for Monte Carlo simulation")
else:
    print("Monte Carlo simulation disabled or no data available")


In [ ]:
# Generate final outputs using the enhanced run_analysis module
from run_analysis import save_outputs, generate_maps

if final_results_df is not None:
    print("💾 Saving enhanced outputs...")
    
    # Save CSV and JSON with all enhanced columns
    save_outputs(final_results_df)
    
    # Generate maps
    generate_maps(final_results_df, gis_data)
    
    print("✅ All outputs saved to 'output' directory")
    print("\nEnhanced files generated:")
    print("  📄 water_risk_scores.csv - Enhanced CSV with all new columns")
    print("  📋 well_metrics.json - Complete JSON metrics")
    print("  🗺️  [API]_map.png - Risk maps for each well")
    
    # Display final summary
    print(f"\n📈 Enhanced Water Safeguarded Summary:")
    summary = final_results_df[['WELL_NAME', 'risk_tier', 'Water_Safeguarded_acft_yr']].copy()
    summary = summary.sort_values('Water_Safeguarded_acft_yr', ascending=False)
    print(summary.to_string(index=False))
    
    total_safeguarded = final_results_df['Water_Safeguarded_acft_yr'].sum()
    print(f"\n🌊 Total Water Safeguarded Across All Wells: {total_safeguarded:.2f} acre-feet/year")
else:
    print("❌ No results to save")
